In [2]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('../01_preprocess/preprocessed.csv')

df.drop(['cycle', 'instr', 'LLC_miss','CPU','L1_miss', 'br_miss', 'scale', 'counted_final_key_count', 'GHz', 'task', 'IPC'], axis=1, inplace=True)
df.head()

,config_name,const_basicHintCount,const_enableBasicHead,const_enableDense,const_enableDense2,const_enableDensifySplit,const_enableHash,const_enableHashAdapt,const_enableHeadNode,const_enablePrefix,...,ycsb_range_len,ycsb_zipf,time,nodeCount_Leaf,nodeCount_Inner,nodeCount_Dense,nodeCount_Hash,nodeCount_Head4,nodeCount_Head8,nodeCount_Dense2
0,hash,16,1,0,0,0,1,0,0,1,...,100,1.133614,2.275,0,846,0,121433,0,0,0
1,hash,16,1,0,0,0,1,0,0,1,...,100,1.133614,1.944,0,846,0,121433,0,0,0
2,dense3,16,1,1,0,1,0,0,0,1,...,100,0.316120,1.529,38703,270,0,0,0,0,0
3,dense3,16,1,1,0,1,0,0,0,1,...,100,0.316120,5.009,38703,270,0,0,0,0,0
4,hints,16,1,0,0,0,0,0,0,1,...,100,0.666485,2.366,27850,742,0,0,0,0,0


In [3]:
# Identify non-numeric columns
non_numeric_columns = df.select_dtypes(['object']).columns

print("Number of columns:", df.shape[1])

# Apply one-hot encoding to non-numeric columns
df = pd.get_dummies(df, columns=non_numeric_columns)

print("Number of columns after one-hot:", df.shape[1])

Number of columns: 35
Number of columns after one-hot: 40


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X = df.drop('time', axis=1)
y = df['time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = gb_regressor.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.029775280518148668


In [9]:
category = pd.qcut(y_train, q=10, labels=False)
def find_category(number):
    
    closest_value = min(category, key=lambda x: abs(x - number))
    return closest_value


In [10]:
# Calculate the predicted category for each data point in the test set
predicted_category = [find_category(prediction) for prediction in y_pred]

# Calculate the actual category for each data point in the test set
cut = pd.qcut(df['time'], q=10, labels=False, duplicates='drop')

# Filter y_test to only include indices present in 'cut'
y_test_filtered = [actual for actual in y_test if actual in cut.index]

actual_category = [cut[actual] for actual in y_test_filtered]

# Calculate the precision
precision = sum([1 for pred, actual in zip(predicted_category, actual_category) if pred == actual]) / len(y_test_filtered)
print("Precision:", precision)


Precision: 0.21052631578947367


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler() 
# Split the data into training and testing sets
X = df.drop('time', axis=1)
y = df['time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler.fit(X_train) 
X_test = scaler.transform(X_test)  
# Train the Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = gb_regressor.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 1.8440661109371408


/home/roland/.cache/pypoetry/virtualenvs/code-RswzgKQQ-py3.11/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
